### Created all parts as functions to allow to be run multiple times with varied parameters

### Import Required Modules 

In [177]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.optimize as opt

#%matplotlib qt
# qt if plot in seperate window
# inline if plot on page

## Set the number of d.p. displayed in numpy arrays
np.set_printoptions(precision=3)

## Progress bar widget
#from IPython.html.widgets import FloatProgress
from IPython.display import display

## allow timing of events
import time

## allow export of results as csv
import csv

In [178]:
### Alpha beta calculator - normal dist

def alphacalc_normal(alphabeta, sd):
    """Return alphabetanew and alpha from normal distribution as specified by sd.
    Default is beta = 0.03
    'alphabeta' is the alphabeta ratio
    If a negative value is returned it is resampled until positive"""
    
    beta = 0.03 # fixed beta in function
    
    ## get alpha beta to use from normal distribution
    if sd == 0:
        alphabetanew = alphabeta
    else:
        alphabetanew=np.random.normal(loc = alphabeta, scale = sd)
    
    ## make sure a positive value is returned
    while alphabetanew <= 0:
        alphabetanew=np.random.normal(loc = alphabeta, scale = sd)
    
    alpha = beta*alphabetanew
   
    return alpha, beta
## alpha/beta can be calced form the returned alpha and beta values

In [179]:
### Alpha beta calculator - log normal dist

def alphacalc_lognormal(alphabeta, sd):
    """Return alphabetanew and alpha from normal distribution as specified by sd.
    Default is beta = 0.03
    'alphabeta' is the alphabeta ratio"""
    
    beta = 0.02 # fixed beta in function
    
    alphabeta_lognormal = np.log((alphabeta**2)/(np.sqrt((sd**2)+(alphabeta**2))))
    sd_lognormal = np.sqrt(np.log(((sd**2)/(alphabeta**2))+1))
    
    ## get alpha beta to use from normal distribution
    if sd == 0:
        alphabetanew = alphabeta
    else:
        alphabetanew=np.random.lognormal(mean = alphabeta_lognormal, sigma = sd_lognormal)
    
    alpha = beta*alphabetanew
   
    return alpha, beta
## alpha/beta can be calced form the returned alpha and beta values

###N0 varies depending on the a/b Std Dev!!!

Need to rethink how to determine N0 in a simple way.

- Ideally re-write entire thing to accept a set of parameters and then use these single parameters to feed into other functions.


**Ideas**
Could use some sort of fitting with all parameters fixed except N0.
Then try and minimise the difference between TCP_calc and TCP_input?
- Can set it to repeat the TCP calc for 1000 patients, then returnt he TCP at d_interest.
- This would use just a single set of nominal parameters (as input by user/in model).
- Only after this does the simulation of multiple parameter variations start.

I cannot think of another way of doing this, as I ahve been doing it manually (trial and error) so far...


In [180]:
## function to calculate the difference between input TCP and calculated TCP. only N0 is varied as want to estimate this.
def calc_dif_sq(x,TCP, n, alphabeta_use, alphabeta_sd_use,d,d_shift,d_sd,d_trend,max_d,dose_of_interest):
    TCP_in = TCP
    
#    TCP_calc_all = completeTCPcalc(n=2000,
#                               alphabeta_use=3,
#                               alphabeta_sd_use=1,
#                               d=2,
#                               d_shift=0,
#                               d_sd=1,
#                               n0=x, # this needs to be able to vary to allow optimisation of the value
#                               max_d=100,
#                               dose_of_interest=74)
    TCP_calc_all = completeTCPcalc(n=n,
                               alphabeta_use=alphabeta_use,
                               alphabeta_sd_use=alphabeta_sd_use,
                               d=d,
                               d_shift=d_shift,
                               d_sd=d_sd,
                               d_trend=d_trend,
                               n0=x, # this needs to be able to vary to allow optimisation of the value
                               max_d=max_d,
                               dose_of_interest=dose_of_interest)
    TCP_result = TCP_calc_all[10] ## Get only the result of interest (N0 is in posn. 10 int he returned tuple)
    
    TCP_Dif_sq = (TCP_in - TCP_result)**2 ## difference in squares to minimise
    return TCP_Dif_sq

In [201]:
## Determine N0 by minimising difference betwen calculate TCP and input TCP.

def n0_determination(TCP_input,
                     repeats,
                     n,
                     alphabeta_use,
                     alphabeta_sd_use,
                     d,
                     d_shift,
                     d_sd,
                     d_trend,
                     max_d,
                     dose_of_interest):
    
    ## TCP to fit N0 to. This will come from the literature.
    print("Fitting N0 Started")
    
    TCP_input = TCP_input

    ## Run optimisation multiple times to ensure accuracy (as based on random data)
    repeats = repeats
    ## store the fit results in this list
    fit_results=[]

    #TCP, n, alphabeta_use, alphabeta_sd_use,d,d_shift,d_sd,max_d,dose_of_interest
    
    for i in range(0,repeats):
        n0_result = opt.minimize_scalar(calc_dif_sq,method='brent',args=(TCP_input,n,alphabeta_use,alphabeta_sd_use,d,d_shift,d_sd,d_trend,max_d,dose_of_interest))
        fit_results.append(n0_result.x) ## build up a list of the N0 fit results
        no_fit_perc_completed = 100*i/repeats  ## Display the progress
        print("\r" + "Fitting N0: " + str(int(no_fit_perc_completed)) + "% complete", end="")

    print("Fitting N0: 100% complete")

    ## Remove outliers from N0 predictions (based on keeping the central 30% of results (within ~0.5 SD))
    trimmed = fit_results[:]
    while len(trimmed)>0.3*len(fit_results):
        trimmed.remove(max(trimmed))
        trimmed.remove(min(trimmed))

    ## Calculate mean of remaining results
    n0_mean_fit=sum(trimmed)/len(trimmed)
    print("N0 mean fit: " + str(n0_mean_fit))

    ## Perform check calculation - not needed for actual use
    TCP_Calc_min = completeTCPcalc(n=n,
                               alphabeta_use=alphabeta_use,
                               alphabeta_sd_use=alphabeta_sd_use,
                               d=d,
                               d_shift=d_shift,
                               d_sd=d_sd,
                               d_trend=d_trend,
                               n0=n0_mean_fit, # this needs to be able to vary to allow optimisation
                               max_d=max_d,
                               dose_of_interest=dose_of_interest)
    print('TCP target: ' + str(TCP_input))
    print('TCP fit check: ' + str(TCP_Calc_min[10]))
    
    return n0_mean_fit, fit_results, trimmed

### a/b SD
This parameter makes a significant difference to the N0 and the TCP values

In [182]:
## N0 determination from entered TCP model parameters - DOES NOT WORK
def n0_determination_old(TCP,d=2,D=74,ab=3,b=0.02):
    """Determine the value of N0 from the input of the
    nominal TCP model input parameters.
    This enables the model to match that of clinical trials data if input."""
    a = b * ab #ab ratio = a/b
    n0 = -np.log(TCP)/np.exp(-(a*D)-(b*d*D))
    return n0

In [183]:
abc = n0_determination_old(0.88,2,74,3,0.02)
print(abc)

209.133405424


In [184]:
tester = np.array([])
tester2 = np.array([])

n2=10000

mean_n = 10
sd_n = 2

### mean and sd required for lognormal calc as described here: http://uk.mathworks.com/help/stats/lognstat.html?refresh=true
mean_l = np.log((mean_n**2)/(np.sqrt((sd_n**2)+(mean_n**2))))
sd_l = np.sqrt(np.log(((sd_n**2)/(mean_n**2))+1))

for i in range(0,n2):
    tester = np.append(tester,[np.random.normal(mean_n,sd_n)])
    tester2 = np.append(tester2,[np.random.lognormal(mean_l,sd_l)])
    
#tester = np.reshape(tester,(n2,12))
#tester2 = np.reshape(tester2,(n2,2))
#print(tester)

print(sd_l)

#plt.hist(tester, bins=30, color='red', alpha=0.5, label='norm')
#plt.hist(tester2, bins=30, color='blue', alpha=0.5, label='lognorm')
#plt.legend()
##print(tester2)
#plt.show()

0.198042200435


In [185]:
###calculate dose for a given fraction based on normal distribution around dose shift
### Log normal distribution not necessary as SD small wrt mean so v unlikely to get negative values returned.
### Could also set a limit on the range (say to +/- 5%)? But wont make much difference with a small SD anyway.

def fracdose(dose, shift, sd):
    """Return dose_actual from normal distribution around dose (Gy) as specified by sd (%) and shift (%).
    Default is dose = 2Gy, shift = 0%, and sd of 0%
    If a negative value is returned it is resampled until positive
    The standard deviation is of the nominal dose"""
    
    ## get actual dose to use from normal distribution based on shift
    
    dose_shift = dose + (dose*shift/100)
    
    ## if sd is zero, then no change to dose
    if sd == 0:
        dose_actual = dose_shift
        return dose_actual
    
    dose_actual=np.random.normal(loc = dose_shift, scale = (dose*sd/100))
    
    ## make sure a positive value is returned
    while dose_actual <= 0:
        dose_actual=np.random.normal(loc = dose_shift, scale = (dose*sd/100))
    
    return dose_actual

In [186]:
## Survival Fraction Calculation
def SFcalc(alpha, beta, dose):
    """Return the SF with input values.
    Note this is for a single dose delivery.
    The product of multiple fractions shoudld be taken
    to give overall SF"""
    
    SF = np.exp(-(alpha*dose) - (beta*(dose**2)))
    
    return SF

In [187]:
## TCP Calculation absed on cumulative SF
def TCPcalc(sf, n0):
    """Return the TCP with input values.
    Based on cumulative SF and N0"""
    
    TCP = np.exp(-n0*sf)
    
    return TCP

In [188]:
## Calc Number of fractions to get to max dose (note: round up as always want an integer)

def no_frac_nom_doses_array(max_d, d):
    n_frac = np.ceil(max_d/d)

    fractions = np.arange(1,n_frac+1)
    #print(fractions)
    nom_doses = np.arange(d,(d*n_frac)+d, step = d)
    #print(nom_doses)
    return fractions, nom_doses, n_frac

In [189]:
## This gives a column with the patient number and makes it easier to check values as going

def create_patients(n):
    
    if n<1:
        n=1
    patients = np.arange(0,n)+1
    patients.shape=(n,1)
    #print(patients)
    return patients

In [190]:
## empty array to store alpha values in (log normal distribution to be used)

def create_alpha_beta_array(n, alphabeta_use, alphabeta_sd_use):
    alpha_and_beta =[]

    for p in range(0,n):
        #alpha_and_beta = np.append(alpha_and_beta,[alphacalc_normal(alphabeta = alphabeta_use, sd=alphabeta_sd_use)])
        alpha_and_beta.append([alphacalc_lognormal(alphabeta = alphabeta_use, sd=alphabeta_sd_use)])

    ## reshape to get a row per patient
    alpha_and_beta_np = np.array(alpha_and_beta)
    alpha_and_beta_np = np.reshape(alpha_and_beta_np,(n,2))
    #print(alpha_and_beta)
    return alpha_and_beta_np

In [191]:
## Calculate Doses for all patients and all fractions and put in an array
## Added in a linear output trend. Specify in percent per day/treatment. Default = 0

def doses_array(n, n_frac, d, d_shift, d_sd, d_trend=0):
    doses = []
    for i in range(0,int(n*n_frac)):
        doses.append(fracdose(dose = d, shift=d_shift, sd=d_sd))
    doses_np = np.array(doses)
    doses_np = np.reshape(doses_np,(n,n_frac))
    
    ## Make an array of the trend to multiply the doses array with
    trend_array = []
    for i in range(int(n_frac)):
        trend_array.append(1+(i*d_trend/100))
        
    ## multiply array of doses by trend value for each fraction    
    doses_np_trend = doses_np*trend_array
        
    #print(doses)
    return doses_np_trend

In [192]:
## Example calc and plot of doses with trend.

x = doses_array(10,100,2,-2,0.5,3/365)
print(x[1])
for i in range(9):
    plt.plot(x[i], alpha=0.5)
    
plt.plot(x[1], linewidth=3, c='black')
plt.show()

[ 1.975  1.962  1.943  1.955  1.969  1.956  1.947  1.943  1.948  1.963
  1.957  1.956  1.969  1.967  1.965  1.95   1.972  1.978  1.968  1.956
  1.972  1.955  1.961  1.963  1.953  1.954  1.971  1.972  1.969  1.962
  1.989  1.967  1.955  1.955  1.971  1.958  1.96   1.971  1.971  1.972
  1.962  1.975  1.966  1.97   1.97   1.974  1.974  1.955  1.971  1.968
  1.961  1.978  1.959  1.977  1.963  1.959  1.98   1.979  1.95   1.964
  1.991  1.961  1.964  1.979  1.96   1.986  1.962  1.981  1.973  1.986
  1.961  1.965  1.965  1.977  1.982  1.983  1.99   1.981  1.98   1.977
  1.961  1.986  1.986  1.982  1.982  1.984  1.992  1.966  1.975  1.968
  1.978  1.98   1.978  1.976  1.959  1.991  1.971  1.963  1.955  1.983]


In [193]:
## Calculate Doses for all patients and all fractions and put in an array

def doses_array_orig(n, n_frac, d, d_shift, d_sd):
    doses = []
    for i in range(0,int(n*n_frac)):
        doses.append(fracdose(dose = d, shift=d_shift, sd=d_sd))
    doses_np = np.array(doses)
    doses_np = np.reshape(doses_np,(n,n_frac))
    #print(doses)
    return doses_np

In [194]:
## Combine all results into single array which may be easier to work with for analysis

def combine_results(patients, alpha_and_beta, doses):
    results_wanted = (patients,alpha_and_beta, doses)
    all_results = np.concatenate(results_wanted, axis=1)
    #print(all_results)
    return all_results

In [195]:
## Loop through the doses of the first patient (first row [0] of array)
def calc_all_SFs(patients, n, n_frac, alpha_and_beta, doses):
    SFs = []

    for i in range(0,len(patients)): # loop through each patient (row)
        for j in range(0,int(n_frac)): # loop through each fraction for each patient (col)
            SFs.append(SFcalc(alpha_and_beta[i][0],alpha_and_beta[i][1],doses[i,j]))

    SFs_np = np.array(SFs)
    SFs_np = np.reshape(SFs_np,(n,n_frac))

    ## GEt cumulative SF for each patient
    SF_cum = np.cumprod(SFs_np, axis=1)
    return SFs_np, SF_cum

In [196]:
## append results to text file.

def saveasCSV(filename, array):

    fl = open(filename, 'a', newline='\n')

    writer = csv.writer(fl)
    writer.writerow(array)

    fl.close()

In [197]:
## Calc Number of fractions and nominal dose per fraction to get to max dose
    
def completeTCPcalc(n,
                   alphabeta_use,
                   alphabeta_sd_use,
                   d,
                   d_shift,
                   d_sd,
                   d_trend,
                   n0,
                   max_d,
                   dose_of_interest):
    
    fractions, nom_doses, n_frac = no_frac_nom_doses_array(max_d,d)

    ## create array containing number of patients in population
    patients = create_patients(n)

    ## Creat array of alpha and veta values for each patient
    alpha_and_beta = create_alpha_beta_array(n, alphabeta_use, alphabeta_sd_use)

    ## array of doses after each fraction for each patient
    doses = doses_array(n, n_frac, d, d_shift, d_sd, d_trend)

    ## put all results in an array with a patient on each row
    all_results = combine_results(patients, alpha_and_beta, doses)

    ## Calc cumulative SF for all patients (also return individual fraction SFs)
    SFs, SF_cum = calc_all_SFs(patients, n, n_frac, alpha_and_beta, doses)

    ## Calculate TCP for all individual patients and fractions
    TCPs = TCPcalc(sf = SF_cum, n0=n0)

    ## Calculate population TCP by averaging down the columns
    TCP_pop = np.mean(TCPs, axis = 0)

    frac_of_interest = dose_of_interest/d

    TCP_at_dose_of_interest = TCP_pop[frac_of_interest]

    #t_end = time.time()

    #t_total = t_end-t_start
    #print(str(round(t_total,2)) + ' secs for ' + str(n) + ' patients')

    TCPs_of_interest = TCPs[:,frac_of_interest-1]

    TCP_cure = (TCPs_of_interest).sum()
    TCP_cure_percent = 100*TCP_cure/n
    
    return n,alphabeta_use,alphabeta_sd_use,d,d_shift,d_sd,n0,max_d,dose_of_interest,frac_of_interest,TCP_cure_percent, TCPs, TCP_pop, nom_doses

In [198]:
## Plot of individual and population TCPs as a function for ease

def TCP_plot(no_ind_plots, label):
    #no_ind_plots = 50

    ## individual plots cannot be more than total patients
    if(no_ind_plots>n):
        no_ind_plots=n

    ## want to select the individual plots randomly from those calcualted...
    ind_plots = np.random.choice(len(TCPs),no_ind_plots, replace=False)

    ## individuals (specified number of plots chosen)
    for i in ind_plots:
        plt.plot(nom_doses,TCPs[i], color = 'grey', alpha = 0.5)
    ## population
    plt.plot(nom_doses,TCP_pop, color='black', linewidth='2', alpha=0.5)
    plt.plot(nom_doses,TCP_pop, marker = 'o', ls='none', label=label)

    ## plot formatting
    plt.xlim(0,max(nom_doses))
    plt.ylim(0,1.0)
    plt.xlabel('Dose (Gy)')
    plt.ylabel('TCP')
    plt.title('TCPs')
    #plt.legend(loc = 'best', fontsize = 'medium', framealpha = 1)
    plt.axvline(d_interest, color = 'black', ls='--',)
    plt.axhline(TCP_pop[frac_interest-1], color='black', ls='--')

    ## add labels with TCP at dose of interest
    text_string = ('Pop. TCP = ' + str(round(TCP_cure_at_d_interest,2)) + ' % at ' + str(d_interest) + 'Gy')
    plt.text(5,0.4,text_string, backgroundcolor='white')
    plt.legend(loc = 'lower left',numpoints=1)

    plt.show()

In [199]:
## Determine N0 by minimising difference betwen calculate TCP and input TCP.

TCP_input = 80 # TCP in percent

## Return sum of squares [calc = calculation of TCP from TCP calc, ref = TCP_input]
def sq_dif(calc, ref):
    sq_dif = (calc - ref)**2
    return sq_dif

### Need to add something to ensure the minimum is always found, but without always having a huge list of possible values.
## Could also start with a coarse list and refine after one sweep?
j=1
start = 0
increment=10

any_pos = False
any_neg = False
stop = False

while (stop == False and j<100):
    TCP_list = []
       
    a_list = list(range(start,(j*increment)))
    print(a_list)
    
    for i in a_list:
        TCP_Calc = completeTCPcalc(n=100,
                               alphabeta_use=3,
                               alphabeta_sd_use=1,
                               d=2,
                               d_shift=0,
                               d_sd=1,
                               d_trend=0,
                               n0=i, # this needs to be able to vary to allow optimisation
                               max_d=100,
                               dose_of_interest=74)
        TCP_list.append(TCP_Calc[10])

    dif_list = np.array(TCP_list)-TCP_input
    dif_list_abs = abs(np.array(TCP_list)-TCP_input)
    print("test")
    print(dif_list)

    any_pos = np.any(dif_list >0)
    any_neg = np.any(dif_list <0)
    
    if (any_pos == True and any_neg == True):
        stop = True
        #break
    
    j = j+1 # increment list to lookup
    start = start + increment
    
print("N0 fitted")
## can use the following to check if the minimum has been found by cehckign for + and - differences.


#dif_list[:] = [x-TCP_input for x in TCP_list]

#a[:] = [x - 13 for x in a]
    
### Now want to 

#tester = sp.optimize.minimize(sq_dif(TCP_Calc(x),TCP_input))
print(min(dif_list_abs)) # minimum difference found.
print(np.argmin(dif_list_abs))# index of minimum found
print("N0: " + str(a_list[np.argmin(dif_list_abs)]))

## So n0 = a_list[np.argmin(dif_list)]

TCP_Calc_min = completeTCPcalc(n=1000,
                           alphabeta_use=3,
                           alphabeta_sd_use=1,
                           d=2,
                           d_shift=0,
                           d_sd=1,
                           d_trend=0,
                           n0=a_list[np.argmin(dif_list_abs)], # this needs to be able to vary to allow optimisation
                           max_d=100,
                           dose_of_interest=74)
print(TCP_Calc_min[10])


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:39: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:46: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


test
[ 20.     19.867  19.73   19.583  19.448  19.431  19.238  19.099  18.984
  18.927]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
test
[ 18.731  18.47   18.172  18.177  18.193  17.878  17.969  18.257  18.228
  17.736]
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
test
[ 17.46   16.727  17.469  17.05   16.347  16.578  16.272  17.03   16.416
  16.609]
[30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
test
[ 15.633  16.517  15.66   15.515  15.56   15.94   15.364  15.847  14.797
  15.7  ]
[40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
test
[ 15.177  13.947  14.364  14.892  14.891  14.744  14.239  14.585  13.959
  13.296]
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
test
[ 13.401  13.561  13.02   12.493  12.036  12.911  12.643  13.419  13.084
  13.053]
[60, 61, 62, 63, 64, 65, 66, 67, 68, 69]
test
[ 12.908  12.753  11.45   12.278  12.288  12.037  11.884  12.143  11.277
  12.038]
[70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
test
[ 11.841  11.272  12.487  12.251  11.779  12.173  12.57   10.021  11.042
  10.319]
[80, 81, 

In [202]:
n0_mean_fit, fit_results, trimmed = n0_determination(TCP_input=80,
                                                     repeats=50,
                                                     n=1000,
                                                     alphabeta_use=3,
                                                     alphabeta_sd_use=1,
                                                     d=2,
                                                     d_shift=0,
                                                     d_sd=1,
                                                     d_trend=0,
                                                     max_d=100,
                                                     dose_of_interest=74)

Fitting N0 Started


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:39: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:46: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Fitting N0: 98% completeFitting N0: 100% complete
N0 mean fit: 200.877815542
TCP target: 80
TCP fit check: 79.3888437953


In [162]:
import matplotlib.pyplot as plt

plt.hist(fit_results)
plt.hist(trimmed)
plt.show()

### Calculate and Plot results of single set of parameters

In [210]:
### Individual calculation of TCP at a set value of dose

n_rpts = 1

print("Enter Dose Shift (%)")
applied_shift = float(input())

text_string = ("Shift: " + str(applied_shift) + "%")

for i in range(0,n_rpts):
    t = completeTCPcalc(n = 1000,
                    alphabeta_use = 3,
                    alphabeta_sd_use = 2,
                    d = 2,
                    d_shift = applied_shift, # +1.6,-2.1% from OP data
                    d_sd = 0,
                    d_trend=0,
                    n0 = n0_mean_fit,
                    max_d = 100,
                    dose_of_interest = 74)

    n=t[0]
    TCP_pop = t[-2]
    TCPs = t[-3]
    nom_doses = t[-1]
    d_interest = t[8]
    frac_interest = t[9]
    TCP_cure_at_d_interest = t[10]



    #print(TCP_pop)
    TCP_plot(100,text_string)

Enter Dose Shift (%)
2


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:39: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:46: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:28: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


### User iterator to build list to vary multiple parameters in one go

In [211]:
## Round to nearest n (e.g. nearest 50 n=50)
def round_n(x,n):
    return int(round(x / n)) * n

In [212]:
## will need to iterate over all the values and add them to a list.

######***** Change this to calcualte a given percentage variaiton in dose? or allow both.....?

## This allows simple building of a tuple containing all possible combinations of values
import itertools

## dose shifts to test
dose_var = 0.5 # dose step size
dose_max = 3
dose_min = -dose_max
dose_number = (dose_max-dose_min)/dose_var+1 # number of points

dose_vals = np.linspace(dose_min,dose_max,dose_number) # dose values to test

## alphabeta values to test
ab_var = 0.5 # step size
ab_max = 4
ab_min = 2
ab_number = (ab_max-ab_min)/ab_var+1 # number of points
ab_vals = np.linspace(ab_min,ab_max,ab_number) # different a/b values to test

## dose of interest to test
di_var = 1 # dose step size
di_max =80
di_min = 64
di_number = (di_max-di_min)/di_var+1 # number of points
di_vals = np.linspace(di_min,di_max,di_number)

## N0 values - determined from TCP model paramters input (from clinical data - coudl get form text file etc?)
#n0_nominal = n0_determination(TCP=0.88,d=2,D=74,ab=3,b=0.02)
n0_nominal = n0_mean_fit
n0_var = 10
n0_range = 5 #percentage variation
n0_min = round_n(n0_nominal * (1-n0_range/100),n0_var)
n0_max = round_n(n0_nominal * (1+n0_range/100),n0_var)
n0_number = (n0_max-n0_min)/n0_var+1
n0_vals = np.linspace(n0_min,n0_max,n0_number)

total_tests = len(dose_vals)*len(ab_vals)*len(di_vals)*len(n0_vals)
test_val_iterator = itertools.product(dose_vals,ab_vals,di_vals,n0_vals)

test_vals = list(test_val_iterator) #list of all combinations of parameters
num_its = len(test_vals) #number of iterations
    
print(num_its)
print(n0_mean_fit)
print(n0_vals)

3315
200.877815542
[ 190.  200.  210.]


In [32]:
#from datetime import datetime

In [214]:
## vary multiple values through use of constructer iterator list above.

#### This needs the N0 to be calculated before calculation starts.
### The values for the iterator above shouls also be calculated within this?
#t1 = datetime.now()

k = 10 # number of repeats

#f = FloatProgress(min=0, max=num_its)
#display(f)

#f1 = FloatProgress(min=0, max=k-1)
#display(f1)

#barpos = 1

all_test_results_array = np.array([]) # array to store all results in before saving as excel csv file
all_test_results_array = []

progress_perc = 0
for j in test_vals:
    
    progress_perc = progress_perc + (100/len(test_vals))
    print('\r' + "TCP Calculation: " + str(int(progress_perc)) + "% completed", end='')
    
    results_array = []
    
    for i in range(0,k):
        t = completeTCPcalc(n = 50,
                        alphabeta_use = j[1],
                        alphabeta_sd_use = 2,
                        d = 2,
                        d_shift = j[0],
                        d_sd = 1,
                        d_trend=0,
                        n0 = j[3],
                        max_d = 100,
                        dose_of_interest = j[2])
        
        #f1.value = i

        n=t[0]
        alphabeta_use = t[1]
        alphabeta_sd_use = t[2]
        d = t[3]
        d_shift = t[4]
        d_sd = t[5]
        n0 = t[6]
        TCP_pop = t[-2]
        TCPs = t[-3]
        nom_doses = t[-1]
        d_interest = t[8]
        frac_interest = t[9]
        TCP_cure_at_d_interest = t[10]
        max_d = 100
        
        results_array.append(TCP_cure_at_d_interest)

        param_array = []
        param_array.append(n)
        param_array.append(k)
        param_array.append(alphabeta_use)
        param_array.append(alphabeta_sd_use)
        param_array.append(d)
        param_array.append(d_shift)
        param_array.append(d_sd)
        param_array.append(n0)
        param_array.append(max_d)
        param_array.append(d_interest)

    ## Array containing results form a single set of parameters (this will update each iteration)
    param_array.extend(results_array)
    param_results_array = param_array
    
    ## Create an array containing all the results for all sets of parameters
    all_test_results_array.extend(param_results_array)
    
    # for updating progress bar
    #barpos = barpos+1
    #f.value = barpos

#print(param_results_array[-1])

## Covnert to Numpy array and reshape so a single set of parameters is on one row
all_test_results_array_np = np.array(all_test_results_array)
all_test_results_array_np = np.reshape(all_test_results_array_np,(num_its,len(all_test_results_array_np)/num_its))

## Save results to file
np.savetxt("TCPresultsProst-all_results0.csv", all_test_results_array_np, delimiter=",",fmt='%10.3f')
print('\r' + "TCP Calculations Completed")

#t2 = datetime.now()
#t3 = (t2-t1).total_seconds()

#print(t3)

TCP Calculation: 0% completed

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:39: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:46: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


TCP Calculations Completed


In [38]:
import time

for i in range(10):
    print('\r' + str(i), end="")
    time.sleep(0.5)

9